In [ ]:
from cycler import cycler
from process_requestlogs import *
from create_window_statistics import *
from gather_memtier_statistics import *
import matplotlib.pyplot as plt

In [ ]:
LOG_INTERVAL = 20 # Every 20th request is logged
WINDOWSIZE_SEC = 5 # The window aggregates over 5 seconds. For the throughput (ops/sec) we need to divide by 5 again
windowsize = str(WINDOWSIZE_SEC) + 's'

In [ ]:
inputdir = "/home/flo/Downloads/exp3/3_1_middleware_baseline_onemw_2017-11-17_202816/readOnly_1vc8workers/1/middleware_04/2017-11-17 223537"
requests = concatenate_requestlogs(
    inputdir,
    "requests")
requests = sort_by_clock(requests)
metrics = extract_metrics(requests)
metrics = metrics.reset_index()
windows = create_window_statistics(metrics, windowsize)
windows['count'] = windows['count'] * LOG_INTERVAL / WINDOWSIZE_SEC

In [ ]:
inputdir = "/home/flo/Downloads/exp3/3_1_middleware_baseline_onemw_2017-11-17_202816/readOnly_1vc8workers/2/middleware_04/2017-11-17 231745"
requests = concatenate_requestlogs(
    inputdir,
    "requests")
requests = sort_by_clock(requests)
metrics2 = extract_metrics(requests)
windows2 = create_window_statistics(metrics2, windowsize)
windows2['count'] = windows2['count'] * LOG_INTERVAL / WINDOWSIZE_SEC

In [ ]:
inputdir = "/home/flo/Downloads/exp3/3_1_middleware_baseline_onemw_2017-11-17_202816/readOnly_1vc8workers/3/middleware_04/2017-11-18 000008"
requests = concatenate_requestlogs(
    inputdir,
    "requests")
requests = sort_by_clock(requests)
metrics3 = extract_metrics(requests)
windows3 = create_window_statistics(metrics3, windowsize)
windows3['count'] = windows3['count'] * LOG_INTERVAL / WINDOWSIZE_SEC

In [ ]:
ping_log_filepath = '/home/flo/Downloads/exp3/3_1_middleware_baseline_onemw_2017-11-17_202816/readOnly_1vc8workers/1/client_ping_01.log'
startingClockTime=metrics['initializeClockTime'].min()
print(startingClockTime)
ping_log = extract_ping_logs(ping_log_filepath, startingClockTime)

In [ ]:
# Throughput of the three runs
fig, ax = plt.subplots()
#ax.set_ylim([0,50000])
#ax.set_xlim([0, 800])
ax.set_prop_cycle(cycler('color',['#a6611a','#dfc27d', '#018571', '#80cdc1']))
ax.plot(windows['initializeClockTime'], windows['count'])
ax.plot(windows2['initializeClockTime'], windows2['count'])
ax.plot(windows3['initializeClockTime'], windows3['count'])
#plt.xticks(grp['num_clients'].head(7))
ax.legend(loc="center right")
ax.set_title("")
ax.set_xlabel("time (sec)")
ax.set_ylabel("Throughput (ops/sec)")

plt.show()

In [ ]:
# Responsetimes of the three runs
fig, ax = plt.subplots()
#ax.set_ylim([0,50000])
#ax.set_xlim([0, 800])
ax.set_prop_cycle(cycler('color',['#a6611a','#dfc27d', '#018571', '#80cdc1']))
ax.plot(windows['initializeClockTime'], windows['responseTime_ms'])
ax.plot(windows2['initializeClockTime'], windows2['responseTime_ms'])
ax.plot(windows3['initializeClockTime'], windows3['responseTime_ms'])
#plt.xticks(grp['num_clients'].head(7))
ax.legend(loc="center right")
ax.set_title("")
ax.set_xlabel("time (sec)")
ax.set_ylabel("Response Time (msec)")

plt.show()

In [ ]:
# Comparison of client throughput with middleware throughput
client_log_filepath='/home/flo/Downloads/exp3/3_1_middleware_baseline_onemw_2017-11-17_202816/readOnly_1vc8workers/1/client_01.log'
client_logs = extract_client_logs(client_log_filepath)

fig, ax = plt.subplots()
ax.set_prop_cycle(cycler('color',['#a6611a','#dfc27d', '#018571', '#80cdc1']))
ax.plot(windows['initializeClockTime'], windows['count'])
ax.plot(client_logs['timestep'], client_logs['throughput'])
#plt.xticks(grp['num_clients'].head(7))
ax.legend(loc="center right")
ax.set_title("")
ax.set_xlabel("time (sec)")
ax.set_ylabel("Response Time (msec)")

plt.show()

fig, ax = plt.subplots()
ax.set_ylim([0,5])
ax.set_xlim([0, 90])
ax.set_prop_cycle(cycler('color',['#a6611a','#dfc27d', '#018571', '#80cdc1']))
ax.plot(windows['initializeClockTime'], windows['responseTime_ms'])
ax.plot(client_logs['timestep'], client_logs['responsetime'])
ax.plot(ping_log['clockTime'], ping_log['rtt'])
#plt.xticks(grp['num_clients'].head(7))
ax.legend(loc="center right")
ax.set_title("")
ax.set_xlabel("time (sec)")
ax.set_ylabel("Response Time (msec)")

plt.show()

In [ ]:
# Histogram of response times
#metrics = metrics.reset_index()
all_data=metrics['responseTime_us']
nowarmup_data=metrics.loc[500:3700, ['responseTime_us']]
binwidth=100
plt.hist(all_data, bins=range(int(all_data.min()), int(all_data.max()) + binwidth, binwidth))

plt.show()